In [19]:
import numpy as np
import pandas as pd
from edward.models import Gamma
import edward as ed
import tensorflow as tf
import matplotlib.pyplot as plt
#from weibull import *
%run weibull.py
%matplotlib inline

In [20]:
P = 1
N = 100
w2 = TruncatedWeibull(shape = 1 * tf.ones(P), scale = 1 * tf.ones(P), cens = 2 * tf.ones(P))

In [22]:
sess = tf.Session()

with sess.as_default():
    y = w2.sample(100).eval().reshape(-1)

In [23]:
y[0:5]

array([ 3.32719088,  2.56861734,  2.04426932,  3.57155252,  2.95205259], dtype=float32)

In [24]:
## Survival times
k = Gamma(alpha = tf.ones(1), beta = tf.ones(1)) # Weibull hyperparameter
p = Gamma(alpha = tf.ones(1), beta = tf.ones(1)) # Weibull hyperparameter

censoring_times = 4 * tf.ones(P)

In [25]:
x = TruncatedWeibull(shape = k * tf.ones(N), scale = p * tf.ones(N), cens = censoring_times)

x2 = TruncatedWeibull2(shape = k * tf.ones(N), scale = p * tf.ones(N))
x2._cens = censoring_times

In [26]:
ds = tf.contrib.distributions
qk = ed.models.TransformedDistribution(
    distribution = ed.models.NormalWithSoftplusSigma(mu = tf.Variable(tf.zeros(1)),
                                                    sigma = tf.Variable(tf.ones(1))),
    bijector = ds.bijector.Exp(),
    name = "qk"
)

qp = ed.models.TransformedDistribution(
    distribution = ed.models.NormalWithSoftplusSigma(mu = tf.Variable(tf.zeros(1)),
                                                    sigma = tf.Variable(tf.ones(1))),
    bijector = ds.bijector.Exp(),
    name = "qp"
)

In [27]:
# %run weibull.py
approx_dict = {
    k: qk,
    p: qp
}

data_dict = {
    x: y
}

inference = ed.KLqp(approx_dict, data = data_dict)

inference.run()

1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: nan


In [28]:
# %run weibull.py
approx_dict = {
    k: qk,
    p: qp
}

data_dict = {
    x2: y
}

inference = ed.KLqp(approx_dict, data = data_dict)

inference.run()

1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: 280.205
